In [3]:
import math

In [1]:
from os import environ, path, walk
from pocketsphinx import *
from sphinxbase import *
import fnmatch
import json
from jiwer import wer
import math

relative_path = "../../td_corpus_digits"
pwd = os.getcwd()
previous = len("1LM")
pwd = pwd[:len(pwd)-previous]
#wer -i SNR35dB_Manrulen.ref SNR35dB_Manrulen.hyp > SNR35dB_Manrulen.res

ModuleNotFoundError: No module named 'pocketsphinx'

In [1]:
def find_files(filename, search_path):
   result = []

   # Wlaking top-down from the root
   for root, dir, files in os.walk(search_path):
      for file in files:
         if file.__contains__(".raw"):        
            result.append(os.path.join(root, file))
   return result

def calculate_CI(wer, noSen):
    return 1.960 * math.sqrt((wer*(1-wer))/noSen)


def create_decoder_ngram(dic, grammar):
    """Create a decoder based on the Ngram language model"""
    config = Decoder.default_config()
    config.set_string('-hmm', pwd +'ps_data/model/en-us')  # acoustic model
    config.set_string('-dict', pwd +'ps_data/lex/' + dic)  # lexicon / dictionary
    config.set_string('-lm', pwd +'ps_data/lm/en-us.lm.bin')  # language model
    decoder_ngram = Decoder(config)
    return decoder_ngram

def create_decoder_digits(dic, grammar, rule):
    config = Decoder.default_config()
    config.set_string('-hmm', pwd +'ps_data/model/en-us')  # acoustic model
    config.set_string('-dict', pwd +'ps_data/lex/' + dic)  # lexicon / dictionary
    decoder_digit = Decoder(config)

    # Now we use a custom language model
    # Prepare the grammar to be used
    jsgf = Jsgf(pwd +'ps_data/jsgf/' + grammar)  # load the grammar file
    rule = jsgf.get_rule('digits.' + rule)  # choose the rule
    fsg = jsgf.build_fsg(rule, decoder_digit.get_logmath(), 7.5)  # build the grammar rule
    fsg.writefile('../outputs/' + grammar)  # write the compiled grammar rule as an external file

    # Now set the fsg grammar rule in the decoder
    decoder_digit.set_fsg("../outputs/" + grammar, fsg)  # load the pre-recorded compiled grammar rule in the decoder
    decoder_digit.set_search("../outputs/" + grammar)  # and set it as the grammar to use

    return decoder_digit

In [4]:
def rundecoder(file_path, decoder):
    # Start the decoder
    decoder.start_utt()

    # Open the file to decode
    stream = open(file_path, 'rb')
    uttbuf = stream.read(-1)

    # Process the file with the decoder
    if uttbuf:
        decoder.process_raw(uttbuf, False, True)
    else:
        print("Error reading speech data")
        exit()
    decoder.end_utt()

    # test for empty hypothesis and replace the output with an empty string if needed
    if decoder.hyp() is None:
        best_hypothesis = ''
    else:
        best_hypothesis = decoder.hyp().hypstr

    #reading ref file for WER
    with open(file_path[:len(file_path)-3] + "ref") as f:
        actualY = f.readlines()
    
    wordErrorRate = wer(actualY, best_hypothesis)

    return {
    "fileName": file_path.split("/")[-1],
    "prediction": best_hypothesis,
    "actual": actualY[0].replace("\n", ""), 
    "confidence": decoder.get_logmath().exp(decoder.hyp().prob),
    "wer" : wordErrorRate}

In [47]:
def processWav(waves, rule, group=""):
    decoder = create_decoder_digits("digits.dict", "digits.jsgf", rule)
    ruleN_result = []
    for wav in waves:
        ruleN_result.append(rundecoder(wav, decoder))

    #this is to dump hyp
    with open(group + rule + '.hyp', 'w') as f:
        for i in ruleN_result:
            f.write(i["prediction"])
            f.write("\n")

    #this is to dump ref
    with open(group + rule + '.ref', 'w') as f:
        for i in ruleN_result:
            f.write(i["actual"])
            f.write("\n")

In [48]:
folder_path = pwd + "td_corpus_digits/SNR35dB/man/seq1digit_200_files/"
wavFiles1 = find_files(".raw",folder_path)

folder_path = pwd + "td_corpus_digits/SNR35dB/man/seq3digits_100_files/"
wavFiles3 = find_files(".raw",folder_path)

folder_path = pwd + "td_corpus_digits/SNR35dB/man/seq5digits_100_files/"
wavFiles5 = find_files(".raw",folder_path)
#Generic ngram

In [ ]:
#ruleN
wavFiles = wavFiles1
wavFiles.extend(wavFiles3)
wavFiles.extend(wavFiles5)
processWav(wavFiles, "rulen", group="SNR35dB_Man")

In [56]:
conf_interval_ruleN = calculate_CI(0.273, 600)
print(conf_interval_ruleN)

0.03564749158075502


In [ ]:
#rule1
processWav(wavFiles1, "rule1", group="SNR35dB_Man")

In [10]:
conf_interval_rule1 = calculate_CI(0.005, 200)
print(conf_interval_rule1*100)

0.9775469298197402


In [ ]:
#rule3
processWav(wavFiles3, "rule3", group="SNR35dB_Man")

In [11]:
conf_interval_rule3 = calculate_CI(0.01667, 100)
print(conf_interval_rule3*100)

2.5094209292536003


In [ ]:
#rule5
processWav(wavFiles5, "rule5", group="SNR35dB_Man")

In [13]:
conf_interval_rule5 = calculate_CI(0.01, 100)
print(conf_interval_rule5*100)

1.950175376728975


In [54]:
def processWavNgram(waves, group=""):
    decoder = create_decoder_ngram("digits.dict", "digits.jsgf")
    ruleN_result = []
    for wav in waves:
        ruleN_result.append(rundecoder(wav, decoder))

    #this is to dump hyp
    with open(group + '.hyp', 'w') as f:
        for i in ruleN_result:
            f.write(i["prediction"])
            f.write("\n")

    #this is to dump ref
    with open(group + '.ref', 'w') as f:
        for i in ruleN_result:
            f.write(i["actual"])
            f.write("\n")

In [ ]:
#NGram
wavFiles = wavFiles1
wavFiles.extend(wavFiles3)
wavFiles.extend(wavFiles5)
processWavNgram(wavFiles, group="SNR35dB_Man")

In [5]:
conf_interval_Ngram = calculate_CI(0.0273, 400)
print(conf_interval_Ngram*100)

1.5969703655359422
